In [1]:
import torch
from torch.utils.data import DataLoader,Dataset,ConcatDataset
import glob
from PIL import Image
import torchvision.transforms.functional as TF
import xml.etree.ElementTree as ET
import numpy as np
from torchvision import transforms

In [2]:
class NimbroSegmentDataset(Dataset):
    def __init__(self,image_dir,image_size):
        self.input_files = glob.glob(image_dir+'image/*.*')
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        self.resize_inp = transforms.Compose([transforms.ToPILImage(),transforms.Resize(image_size),transforms.ToTensor()])
        self.resize_op = transforms.Compose([transforms.ToPILImage(),transforms.Resize((image_size[0]//4,image_size[1]//4)),transforms.ToTensor()])
        self.data = [ i for i in range(len(self.input_files))]
    def __getitem__(self,index):
        f = self.input_files[index]

        if type(self.data[index]) != int:
            target_ = self.data[index]
        else:
            target_path = f.replace('image','target').replace('jpg','png')
            target = Image.open(target_path)
            target_single_channel = TF.to_tensor(target)
            target_single_channel = self.resize_op(target_single_channel)


            target_ = torch.zeros_like(target_single_channel)
            target_[target_single_channel.min() == target_single_channel] = 0

            target_[(target_single_channel.min() < target_single_channel) & (target_single_channel.max() > target_single_channel)] = 2

            target_[target_single_channel.max() == target_single_channel] = 1
            target_ = target_.squeeze()
#             target = torch.stack([target_0.squeeze(),target_1.squeeze(),target_2.squeeze()])
            self.data[index] = target_


        x = Image.open(f)
        x = TF.to_tensor(x)     
        x = self.resize_inp(x)
        x = self.normalize(x)
        
        return x,target_
    def __len__(self):
        return len(self.input_files)